In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"

In [4]:
cd /media/datastorage/Phong/fashion_mnist

/media/datastorage/Phong/fashion_mnist


In [ ]:
# from PIL import Image
# import numpy

# def convert_img_square(im_pth='', dest_path='', desired_size=224):
# #     print(im_pth)
    
#     im = Image.open(im_pth)
#     old_size = im.size  # (width, height) format

#     ratio = float(desired_size)/max(old_size)
#     new_size = tuple([int(x*ratio) for x in old_size])

# #     new_im = im.resize(new_size, Image.ANTIALIAS)
#     new_im = im.resize(new_size)
    
#     new_im = new_im.convert('RGB')
    
#     new_im.save(dest_path)

#     return True

In [ ]:
# ####=======================
# import glob
# import os
# import shutil
# import random

# #move class folder from classname_# to classname/#

# def get_image_parts(image_path):
#     """Given a full path to an image, return its parts."""
#     parts = image_path.split(os.path.sep)
#     #print(parts)
#     filename = parts[2]
#     filename_no_ext = filename.split('.')[0]
#     classname = parts[1]
#     train_or_test = parts[0]
    
#     return train_or_test, classname, filename_no_ext, filename

# move_folders = ['fashion_mnist_train_resized_299']
# dest_folder = 'fashion_mnist_train_resized_350'
# data_file = []

# # look for all images in sub-folders
# for folder in move_folders:
#     class_folders = glob.glob(os.path.join(folder, '*'))
#     print('folder %s' %class_folders)
    
# #     for sub_folder in class_folders:
# #         sub_class_folders = glob.glob(os.path.join(sub_folder, '*'))    
# #         print('sub folder %s' %sub_class_folders)
        
#     for iid_class in class_folders:
#         print(iid_class)
        
#         class_files = glob.glob(os.path.join(iid_class, '*.png'))
        
# #         #Determize Set# (No Suffle)
#         set = len(class_files)
#         inner = range(0*set, 1*set) #all

#         print('moving %d files' %(len(inner)))

# #         random_list = random.sample(range(len(class_files)), int(len(class_files)/5)) #1/5 dataset
# #         for idx in range(len(random_list)):

#         for idx in range(len(inner)):
#             src = class_files[inner[idx]]

#             train_or_test, classname, filename_no_ext, filename = get_image_parts(src)
#             dst = os.path.join(dest_folder, classname, filename)

#             # image directory
#             img_directory = os.path.join(dest_folder, classname)

#             # create folder if not existed
#             if not os.path.exists(img_directory):
#                 os.makedirs(img_directory)
                
#             # convert image
#             convert_img_square(src, dst, 350)
# #             #moving file
# #             shutil.move(src, dst)
# # #                 shutil.copy(src, dst)


In [ ]:
# ####=======================
# import glob
# import os
# import shutil
# import random

# #move class folder from classname_# to classname/#

# def get_image_parts(image_path):
#     """Given a full path to an image, return its parts."""
#     parts = image_path.split(os.path.sep)
#     #print(parts)
#     filename = parts[2]
#     filename_no_ext = filename.split('.')[0]
#     classname = parts[1]
#     train_or_test = parts[0]
    
#     return train_or_test, classname, filename_no_ext, filename

# move_folders = ['fashion_mnist_test_resized_299']
# dest_folder = 'fashion_mnist_test_resized_350'
# data_file = []

# # look for all images in sub-folders
# for folder in move_folders:
#     class_folders = glob.glob(os.path.join(folder, '*'))
#     print('folder %s' %class_folders)
    
# #     for sub_folder in class_folders:
# #         sub_class_folders = glob.glob(os.path.join(sub_folder, '*'))    
# #         print('sub folder %s' %sub_class_folders)
        
#     for iid_class in class_folders:
#         print(iid_class)
        
#         class_files = glob.glob(os.path.join(iid_class, '*.png'))
        
# #         #Determize Set# (No Suffle)
#         set = len(class_files)
#         inner = range(0*set, 1*set) #all

#         print('moving %d files' %(len(inner)))

# #         random_list = random.sample(range(len(class_files)), int(len(class_files)/5)) #1/5 dataset
# #         for idx in range(len(random_list)):

#         for idx in range(len(inner)):
#             src = class_files[inner[idx]]

#             train_or_test, classname, filename_no_ext, filename = get_image_parts(src)
#             dst = os.path.join(dest_folder, classname, filename)

#             # image directory
#             img_directory = os.path.join(dest_folder, classname)

#             # create folder if not existed
#             if not os.path.exists(img_directory):
#                 os.makedirs(img_directory)
                
#             # convert image
#             convert_img_square(src, dst, 299)
# #             #moving file
# #             shutil.move(src, dst)
# # #                 shutil.copy(src, dst)


In [5]:
## fix for multi_gpu_model prediction time longer
from keras.layers import Lambda, concatenate
from keras import Model
import tensorflow as tf

def multi_gpu_model(model, gpus):
    if isinstance(gpus, (list, tuple)):
        num_gpus = len(gpus)
        target_gpu_ids = gpus
    else:
        num_gpus = gpus
        target_gpu_ids = range(num_gpus)

    def get_slice(data, i, parts):
        shape = tf.shape(data)
        batch_size = shape[:1]
        input_shape = shape[1:]
        step = batch_size // parts
        if i == num_gpus - 1:
            size = batch_size - step * i
        else:
            size = step
        size = tf.concat([size, input_shape], axis=0)
        stride = tf.concat([step, input_shape * 0], axis=0)
        start = stride * i
        return tf.slice(data, start, size)

    all_outputs = []
    for i in range(len(model.outputs)):
        all_outputs.append([])

    # Place a copy of the model on each GPU,
    # each getting a slice of the inputs.
    for i, gpu_id in enumerate(target_gpu_ids):
        with tf.device('/gpu:%d' % gpu_id):
            with tf.name_scope('replica_%d' % gpu_id):
                inputs = []
                # Retrieve a slice of the input.
                for x in model.inputs:
                    input_shape = tuple(x.get_shape().as_list())[1:]
                    slice_i = Lambda(get_slice,
                                   output_shape=input_shape,
                                   arguments={'i': i,
                                              'parts': num_gpus})(x)
                    inputs.append(slice_i)

                # Apply model on slice
                # (creating a model replica on the target device).
                outputs = model(inputs)
                if not isinstance(outputs, list):
                    outputs = [outputs]

                # Save the outputs for merging back together later.
                for o in range(len(outputs)):
                    all_outputs[o].append(outputs[o])

    # Merge outputs on CPU.
    with tf.device('/cpu:0'):
        merged = []
        for name, outputs in zip(model.output_names, all_outputs):
            merged.append(concatenate(outputs,
                                    axis=0, name=name))
        return Model(model.inputs, merged)


Using TensorFlow backend.


In [6]:
from keras.callbacks import Callback
import pickle
import sys

#Stop training on val_acc
class EarlyStoppingByAccVal(Callback):
    def __init__(self, monitor='val_acc', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current >= self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping" % epoch)
            self.model.stop_training = True

#Save large model using pickle formate instead of h5            
class SaveCheckPoint(Callback):
    def __init__(self, model, dest_folder):
        super(Callback, self).__init__()
        self.model = model
        self.dest_folder = dest_folder
        
        #initiate
        self.best_val_acc = 0
        self.best_val_loss = sys.maxsize #get max value
          
    def on_epoch_end(self, epoch, logs={}):
        val_acc = logs['val_acc']
        val_loss = logs['val_loss']

        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            
            # Save weights in pickle format instead of h5
            print('\nSaving val_acc %f at %s' %(self.best_val_acc, self.dest_folder))
            weigh= self.model.get_weights()

            #now, use pickle to save your model weights, instead of .h5
            #for heavy model architectures, .h5 file is unsupported.
            fpkl= open(self.dest_folder, 'wb') #Python 3
            pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
            fpkl.close()
            
#             model.save('tmp.h5')
        elif val_acc == self.best_val_acc:
            if val_loss < self.best_val_loss:
                self.best_val_loss=val_loss
                
                # Save weights in pickle format instead of h5
                print('\nSaving val_acc %f at %s' %(self.best_val_acc, self.dest_folder))
                weigh= self.model.get_weights()

                #now, use pickle to save your model weights, instead of .h5
                #for heavy model architectures, .h5 file is unsupported.
                fpkl= open(self.dest_folder, 'wb') #Python 3
                pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
                fpkl.close()                    

In [9]:
#SET 1

from classification_models.keras import Classifiers
from keras.models import Model
# from keras.layers import concatenate
from keras.layers import Dense, GlobalAveragePooling2D, Input, Embedding, LSTM, SimpleRNN, Reshape, Concatenate,Bidirectional
# from keras.applications.inception_v3 import preprocess_input

from keras.layers import GaussianNoise

senet, preprocess_input = Classifiers.get('seresnext101')

f2_base = senet(input_shape=(299,299,3), weights='imagenet', include_top=False)
f2_x = GlobalAveragePooling2D()(f2_base.output)

f2_x = Reshape([1,2048])(f2_x)  
f2_x = SimpleRNN(2048, 
            return_sequences=False,                       
#             dropout=0.8                                     
            input_shape=[1,2048])(f2_x)

#Regularization with noise
f2_x = GaussianNoise(0.1)(f2_x)

f2_x = Dense(2048, activation='relu')(f2_x)
f2_x = Dense(10, activation='softmax')(f2_x)
model_2 = Model(inputs=[f2_base.input],outputs=[f2_x])

print(model_2.summary())
# # print(f2_base.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_71 (ZeroPadding2 (None, 305, 305, 3)  0           input[0][0]                      
__________________________________________________________________________________________________
conv2d_2387 (Conv2D)            (None, 150, 150, 64) 9408        zero_padding2d_71[0][0]          
__________________________________________________________________________________________________
batch_normalization_209 (BatchN (None, 150, 150, 64) 256         conv2d_2387[0][0]                
__________________________________________________________________________________________________
activation

activation_379 (Activation)     (None, 19, 19, 512)  0           batch_normalization_240[0][0]    
__________________________________________________________________________________________________
zero_padding2d_82 (ZeroPadding2 (None, 21, 21, 512)  0           activation_379[0][0]             
__________________________________________________________________________________________________
lambda_2476 (Lambda)            (None, 21, 21, 16)   0           zero_padding2d_82[0][0]          
__________________________________________________________________________________________________
lambda_2477 (Lambda)            (None, 21, 21, 16)   0           zero_padding2d_82[0][0]          
__________________________________________________________________________________________________
lambda_2478 (Lambda)            (None, 21, 21, 16)   0           zero_padding2d_82[0][0]          
__________________________________________________________________________________________________
lambda_247

lambda_2750 (Lambda)            (None, 21, 21, 16)   0           zero_padding2d_90[0][0]          
__________________________________________________________________________________________________
lambda_2751 (Lambda)            (None, 21, 21, 16)   0           zero_padding2d_90[0][0]          
__________________________________________________________________________________________________
lambda_2752 (Lambda)            (None, 21, 21, 16)   0           zero_padding2d_90[0][0]          
__________________________________________________________________________________________________
lambda_2753 (Lambda)            (None, 21, 21, 16)   0           zero_padding2d_90[0][0]          
__________________________________________________________________________________________________
lambda_2754 (Lambda)            (None, 21, 21, 16)   0           zero_padding2d_90[0][0]          
__________________________________________________________________________________________________
lambda_275

                                                                 conv2d_3394[0][0]                
                                                                 conv2d_3395[0][0]                
__________________________________________________________________________________________________
batch_normalization_295 (BatchN (None, 19, 19, 512)  2048        concatenate_94[0][0]             
__________________________________________________________________________________________________
activation_470 (Activation)     (None, 19, 19, 512)  0           batch_normalization_295[0][0]    
__________________________________________________________________________________________________
conv2d_3396 (Conv2D)            (None, 19, 19, 1024) 524288      activation_470[0][0]             
__________________________________________________________________________________________________
batch_normalization_296 (BatchN (None, 19, 19, 1024) 4096        conv2d_3396[0][0]                
__________

In [10]:
#Non-Groups
#Split training and validation
#Using Expert Data

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras.optimizers import Adam
# from keras.utils import multi_gpu_model

import time, os
from math import ceil
import multiprocessing

NUM_GPU = 4
batch_size = 40
#Using multiple models if more than 1 GPU
if NUM_GPU != 1:
    model_mul = multi_gpu_model(model_2, gpus=NUM_GPU)

epochs = 40##!!!
lr = 1e-4
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)

if NUM_GPU != 1:
    model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
#     rescale = 1./255
    preprocessing_function=preprocess_input
)

train_set = train_datagen.flow_from_directory('fashion_mnist_train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

valid_set = test_datagen.flow_from_directory('fashion_mnist_test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

model_txt = 'bi-lstm'
# Helper: Save the model.
savedfilename = os.path.join('checkpoints', 'FashionMNIST_SEResNext101_LRG299_Mul_RNN.pkl')

earlystopping = EarlyStoppingByAccVal(monitor='val_acc', value=0.99, verbose=1)
savecheckpoint = SaveCheckPoint(model_2, savedfilename)

# Helper: TensorBoard
tb = TensorBoard(log_dir=os.path.join('fmnist_output', 'logs', model_txt))

# Helper: Save results.
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('fmnist_output', 'logs', model_txt + '-' + 'training-' + \
    str(timestamp) + '.log'))

# number of steps each epoch
step_size_train=ceil(train_set.n/train_set.batch_size)
step_size_valid=ceil(valid_set.n/valid_set.batch_size)
# step_size_test=ceil(testing_set.n//testing_set.batch_size)

result = model_mul.fit_generator(
    generator = train_set, 
    steps_per_epoch = step_size_train,
    validation_data = valid_set,
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
    callbacks=[earlystopping, savecheckpoint],
#     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
    verbose=1) 

Found 60000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.
Epoch 1/40
1499/1500 [============================>.] - ETA: 1s - loss: 0.4939 - acc: 0.8223
Saving val_acc 0.900800 at checkpoints/FashionMNIST_SEResNext101_LRG299_Mul_RNN.pkl
1500/1500 [==============================] - 1618s 1s/step - loss: 0.4937 - acc: 0.8224 - val_loss: 0.2795 - val_acc: 0.9008
Epoch 2/40
1499/1500 [============================>.] - ETA: 0s - loss: 0.3263 - acc: 0.8831
Saving val_acc 0.903000 at checkpoints/FashionMNIST_SEResNext101_LRG299_Mul_RNN.pkl
1500/1500 [==============================] - 1562s 1s/step - loss: 0.3263 - acc: 0.8831 - val_loss: 0.2652 - val_acc: 0.9030
Epoch 3/40
1499/1500 [============================>.] - ETA: 0s - loss: 0.2831 - acc: 0.8973
Saving val_acc 0.917600 at checkpoints/FashionMNIST_SEResNext101_LRG299_Mul_RNN.pkl
1500/1500 [==============================] - 1561s 1s/step - loss: 0.2832 - acc: 0.8973 - val_loss: 0.2332 - val_acc: 0.9176
Epoch 

In [11]:
#use pickle to load model weights
import os
import pickle

pklfile= os.path.join('checkpoints', 'FashionMNIST_SEResNext101_LRG299_Mul_RNN.pkl')

f= open(pklfile, 'rb')     #Python 3                 
weigh= pickle.load(f);                
f.close();

# #use set_weights to load the modelweights into the model architecture
# NUM_GPU = 4
# #Using multiple models if more than 1 GPU
# if NUM_GPU != 1:
#     model_mul = multi_gpu_model(model_2, gpus=NUM_GPU)

model_mul.set_weights(weigh)

In [ ]:
mkdir checkpoints

In [12]:
# Change LR=1e-5

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras.optimizers import Adam
# from keras.utils import multi_gpu_model

import time, os
from math import ceil
import multiprocessing

NUM_GPU = 4
batch_size = 40

epochs = 15##!!!
lr = 1e-5
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)

if NUM_GPU != 1:
    model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
#     rescale = 1./255
    preprocessing_function=preprocess_input
)

train_set = train_datagen.flow_from_directory('fashion_mnist_train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

valid_set = test_datagen.flow_from_directory('fashion_mnist_test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

model_txt = 'bi-lstm'
# Helper: Save the model.
savedfilename = os.path.join('checkpoints', 'FashionMNIST_SEResNext101_LRG299_Mul_RNN_L2.pkl')

earlystopping = EarlyStoppingByAccVal(monitor='val_acc', value=0.99, verbose=1)
savecheckpoint = SaveCheckPoint(model_2, savedfilename)

# Helper: TensorBoard
tb = TensorBoard(log_dir=os.path.join('fmnist_output', 'logs', model_txt))

# Helper: Save results.
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('fmnist_output', 'logs', model_txt + '-' + 'training-' + \
    str(timestamp) + '.log'))

# number of steps each epoch
step_size_train=ceil(train_set.n/train_set.batch_size)
step_size_valid=ceil(valid_set.n/valid_set.batch_size)
# step_size_test=ceil(testing_set.n//testing_set.batch_size)

result = model_mul.fit_generator(
    generator = train_set, 
    steps_per_epoch = step_size_train,
    validation_data = valid_set,
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
    callbacks=[earlystopping, savecheckpoint],
#     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
    verbose=1) 

Found 60000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.
Epoch 1/15
1499/1500 [============================>.] - ETA: 1s - loss: 0.0852 - acc: 0.9682
Saving val_acc 0.951600 at checkpoints/FashionMNIST_SEResNext101_LRG299_Mul_RNN_L2.pkl
1500/1500 [==============================] - 1613s 1s/step - loss: 0.0852 - acc: 0.9683 - val_loss: 0.1636 - val_acc: 0.9516
Epoch 2/15
1499/1500 [============================>.] - ETA: 0s - loss: 0.0716 - acc: 0.9737
Saving val_acc 0.951700 at checkpoints/FashionMNIST_SEResNext101_LRG299_Mul_RNN_L2.pkl
1500/1500 [==============================] - 1551s 1s/step - loss: 0.0716 - acc: 0.9738 - val_loss: 0.1702 - val_acc: 0.9517
Epoch 3/15
1499/1500 [============================>.] - ETA: 0s - loss: 0.0684 - acc: 0.9756
Saving val_acc 0.951900 at checkpoints/FashionMNIST_SEResNext101_LRG299_Mul_RNN_L2.pkl
1500/1500 [==============================] - 1554s 1s/step - loss: 0.0684 - acc: 0.9756 - val_loss: 0.1749 - val_acc: 0.95

In [ ]:
max(result.history['val_acc']),min(result.history['val_acc'])

In [ ]:
# Visualize training history
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy

# list all data in history
# history = result.history
#print(history.history.keys())
# summarize history for accuracy
plt.plot(result.history['acc'])
plt.plot(result.history['val_acc'])
plt.title('SeNet - 501 - Noise model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(result.history['loss'])
plt.plot(result.history['val_loss'])
plt.title('SeNet - 501 - Noise model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [13]:
#use pickle to load model weights
import os
import pickle

pklfile= os.path.join('checkpoints', 'FashionMNIST_SEResNext101_LRG299_Mul_RNN_L2.pkl')

f= open(pklfile, 'rb')     #Python 3                 
weigh= pickle.load(f);                
f.close();

# #use set_weights to load the modelweights into the model architecture
# NUM_GPU = 4
# #Using multiple models if more than 1 GPU
# if NUM_GPU != 1:
#     model_mul = multi_gpu_model(model_2, gpus=NUM_GPU)

model_mul.set_weights(weigh)

In [14]:
# Change LR=1e-6

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras.optimizers import Adam
# from keras.utils import multi_gpu_model

import time, os
from math import ceil
import multiprocessing

NUM_GPU = 4
batch_size = 40

epochs = 15##!!!
lr = 1e-6
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)

if NUM_GPU != 1:
    model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
#     rescale = 1./255
    preprocessing_function=preprocess_input
)

train_set = train_datagen.flow_from_directory('fashion_mnist_train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

valid_set = test_datagen.flow_from_directory('fashion_mnist_test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

model_txt = 'bi-lstm'
# Helper: Save the model.
savedfilename = os.path.join('checkpoints', 'FashionMNIST_SEResNext101_LRG299_Mul_RNN_L3.pkl')

earlystopping = EarlyStoppingByAccVal(monitor='val_acc', value=0.99, verbose=1)
savecheckpoint = SaveCheckPoint(model_2, savedfilename)

# Helper: TensorBoard
tb = TensorBoard(log_dir=os.path.join('fmnist_output', 'logs', model_txt))

# Helper: Save results.
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('fmnist_output', 'logs', model_txt + '-' + 'training-' + \
    str(timestamp) + '.log'))

# number of steps each epoch
step_size_train=ceil(train_set.n/train_set.batch_size)
step_size_valid=ceil(valid_set.n/valid_set.batch_size)
# step_size_test=ceil(testing_set.n//testing_set.batch_size)

result = model_mul.fit_generator(
    generator = train_set, 
    steps_per_epoch = step_size_train,
    validation_data = valid_set,
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
    callbacks=[earlystopping, savecheckpoint],
#     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
    verbose=1) 

Found 60000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.
Epoch 1/15
1499/1500 [============================>.] - ETA: 1s - loss: 0.0399 - acc: 0.9854
Saving val_acc 0.952600 at checkpoints/FashionMNIST_SEResNext101_LRG299_Mul_RNN_L3.pkl
1500/1500 [==============================] - 1644s 1s/step - loss: 0.0399 - acc: 0.9854 - val_loss: 0.2082 - val_acc: 0.9526
Epoch 2/15
1500/1500 [==============================] - 1575s 1s/step - loss: 0.0399 - acc: 0.9854 - val_loss: 0.2093 - val_acc: 0.9522
Epoch 3/15
1500/1500 [==============================] - 1574s 1s/step - loss: 0.0399 - acc: 0.9857 - val_loss: 0.2105 - val_acc: 0.9522
Epoch 4/15
1500/1500 [==============================] - 1574s 1s/step - loss: 0.0395 - acc: 0.9852 - val_loss: 0.2115 - val_acc: 0.9524
Epoch 5/15
1500/1500 [==============================] - 1577s 1s/step - loss: 0.0392 - acc: 0.9858 - val_loss: 0.2129 - val_acc: 0.9519
Epoch 6/15
1500/1500 [==============================] - 1575s 1

In [ ]:
result = model_mul.fit_generator(
    generator = train_set, 
    steps_per_epoch = step_size_train,
    validation_data = valid_set,
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
    callbacks=[earlystopping, savecheckpoint],
#     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
    verbose=1) 

In [ ]:
#use pickle to load model weights
import os
import pickle

pklfile= os.path.join('checkpoints', 'FashionMNIST_SEResNext101_LRG299_Mul_LSTM_L3.pkl')

f= open(pklfile, 'rb')     #Python 3                 
weigh= pickle.load(f);                
f.close();

#use set_weights to load the modelweights into the model architecture
NUM_GPU = 4

#Using multiple models if more than 1 GPU
if NUM_GPU != 1:
    model_mul = multi_gpu_model(model_2, gpus=NUM_GPU)

model_mul.set_weights(weigh)

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import time, os
from math import ceil

# PREDICT ON OFFICIAL TEST
train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
#     rescale = 1./255
    preprocessing_function=preprocess_input
)

test_datagen1 = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

batch_size = 40

train_set = train_datagen.flow_from_directory('fashion_mnist_train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

test_set1 = test_datagen1.flow_from_directory('fashion_mnist_test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

if NUM_GPU != 1:
    predict1=model_mul.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
# else:
#     predict1=model.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
    
predicted_class_indices=np.argmax(predict1,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions1 = [labels[k] for k in predicted_class_indices]

import pandas as pd

filenames=test_set1.filenames
results=pd.DataFrame({"file_name":filenames,
                      "predicted1":predictions1,
                      })
results.to_csv('FashionMNIST_SeResNet101_MulModels299_2310_01.csv')
results.head()

In [ ]:
results.tail()

In [ ]:
cp FashionMNIST_SeResNet101_MulModels299_2310_01.csv /home/bribeiro/Phong/Nat19/FashionMNIST_SeResNet101_MulModels299_2310_01.csv

In [ ]:
np.save(os.path.join('pred_npy','FashionMNIST_SEResNext101_LRG299_Mul.npy'), predict1)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from math import ceil
import numpy as np

batch_size = 30

#Crop-Official Test
def random_crop(img, random_crop_size):
    # Note: image_data_format is 'channel_last'
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]

def crop_generator(batches, crop_length):
    """Generate random crops from the image batches"""
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)

test_datagen_crop = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

testing_set_crop = test_datagen_crop.flow_from_directory('fashion_mnist_test_resized_299',
                                                 target_size = (320, 320),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="training"
                                              )
#customized generator
test_crops = crop_generator(testing_set_crop, 299)

step_size_test_crop = ceil(testing_set_crop.n/testing_set_crop.batch_size)

tta_steps = 21
predictions = []

# import tensorflow as tf
# with tf.device('/gpu:0'):
for i in range(tta_steps):
    print(i)
    testing_set_crop.reset()
    if NUM_GPU != 1:
        preds=model_mul.predict_generator(test_crops, 
                                           steps = step_size_test_crop,
#                                            max_queue_size=16,
#                                                use_multiprocessing=True,
#                                            workers=1,
                                           verbose=1)    
#     else:
#         preds=model.predict_generator(test_crops, 
#                                            steps = step_size_test_crop,
#                                            max_queue_size=16,
# #                                                use_multiprocessing=True,
#                                            workers=1,
#                                            verbose=1)  
#     preds=model_2.predict_generator(test_crops,steps = step_size_test_crop,verbose=1)  
    predictions.append(preds)

mean_pred = np.mean(predictions, axis=0)

predicted_class_indices_mean=np.argmax(mean_pred,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
finalpre = [labels[k] for k in predicted_class_indices_mean]

import pandas as pd
filenames=testing_set_crop.filenames
results=pd.DataFrame({"id":filenames,
                      "predicted":finalpre,
                      })
results.to_csv('FashionMNIST_SeResNet101_MulModels299_2310_tta_3_5.csv')
results.head(10)

In [ ]:
cp FashionMNIST_SeResNet101_MulModels299_2310_tta_3_5.csv /home/bribeiro/Phong/Nat19/FashionMNIST_SeResNet101_MulModels299_2310_tta_3_5.csv

In [ ]:
# Save weights in pickle format instead of h5
import pickle

#use get_weights() command to get your model weights
weigh= model_2.get_weights()

#now, use pickle to save your model weights, instead of .h5
#for heavy model architectures, .h5 file is unsupported.
pklfile= os.path.join('checkpoints', 'FashionMNIST_SEResNext101_LRG501_Mul_STD.pkl')

fpkl= open(pklfile, 'wb') #Python 3
pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
fpkl.close()

In [ ]:
os.listdir("checkpoints")

In [ ]:
# #use pickle to load model weights
# import os
# import pickle

# pklfile= os.path.join('checkpoints', 'FashionMNIST_SEResNext101_LRG299_Mul_LSTM_L3.pkl')

# f= open(pklfile, 'rb')     #Python 3                 
# weigh= pickle.load(f);                
# f.close();

# #use set_weights to load the modelweights into the model architecture
# NUM_GPU = 4
# #Using multiple models if more than 1 GPU
# if NUM_GPU != 1:
#     model_mul = multi_gpu_model(model_2, gpus=NUM_GPU)

# model_mul.set_weights(weigh)

In [ ]:
# model 2 - SeResNext101
#use pickle to load model weights
import os
import pickle

# load to multi gpu
NUM_GPU = 4
model_mul = multi_gpu_model(model_2, gpus=NUM_GPU)

#load model the saved weights
pklfile= os.path.join('checkpoints', 'FashionMNIST_SEResNext101_LRG299_Mul_STD_L2.pkl')

f= open(pklfile, 'rb')     #Python 3                 
weigh= pickle.load(f);                
f.close();

#set the saved weight to mul-gpus model
model_mul.set_weights(weigh)

f2_sgl = model_mul.layers[-2]

#get weight from single gpu
weigh= f2_sgl.get_weights()

#now, use pickle to save model weights, instead of .h5
#for heavy model architectures, .h5 file is unsupported.
pklfile= os.path.join('checkpoints', 'FashionMNIST_SEResNext101_LRG299_Mul_STD_L2_SGL.pkl')

fpkl= open(pklfile, 'wb') #Python 3
pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
fpkl.close()

In [ ]:
#model 2 - SEResNext101
#load model the saved weights
pklfile= os.path.join('checkpoints', 'FashionMNIST_SEResNext101_LRG299_Mul_STD_L2_SGL.pkl')

f= open(pklfile, 'rb')     #Python 3                 
weigh= pickle.load(f);                
f.close();

#set the saved weight to mul-gpus model
model_2.set_weights(weigh)

In [ ]:
# # load to multi gpu
# NUM_GPU = 4
# model_mul = multi_gpu_model(model_2, gpus=NUM_GPU)

# #get weight from mul gpu
# weigh= model_mul.get_weights()

# #now, use pickle to save model weights, instead of .h5
# #for heavy model architectures, .h5 file is unsupported.
# pklfile= os.path.join('checkpoints', 'FashionMNIST_SEResNext101_LRG299_Mul_LSTM_L3_SGL.pkl')

# fpkl= open(pklfile, 'wb') #Python 3
# pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
# fpkl.close()

In [ ]:
#Set 1
#MUL 1 - Xception

# from keras.applications import InceptionV3
from keras.applications import Xception
# from keras.applications.inception_resnet_v2 import InceptionResNetV2

from keras.models import Model
from keras.layers import concatenate
from keras.layers import Dense, GlobalAveragePooling2D, Input, Embedding, LSTM, Reshape, Bidirectional
# from keras.applications.inception_v3 import preprocess_input
from keras.applications.xception import preprocess_input

from keras.layers import GaussianNoise

f1_base = Xception(weights='imagenet', include_top=False, input_shape=(299,299,3))  
f1_x = f1_base.output
f1_x = GlobalAveragePooling2D()(f1_x)

# f1_x = Reshape([1,2048])(f1_x)  
# f1_x = Bidirectional(LSTM(1024, 
#                                  return_sequences=False, 
# #                                  dropout=0.8
#                                 ),
#                             input_shape=[1,2048],
#                             merge_mode='concat')(f1_x)

#Regularization with noise
f1_x = GaussianNoise(0.1)(f1_x)

f1_x = Dense(2048, activation='relu')(f1_x)
f1_x = Dense(10, activation='softmax')(f1_x)
model_1 = Model(inputs=[f1_base.input],outputs=[f1_x])

print(model_1.summary())

In [ ]:
#Non-Groups
#Split training and validation
#Using Expert Data

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras.optimizers import Adam
# from keras.utils import multi_gpu_model

import time, os
from math import ceil
import multiprocessing

train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
#     rescale = 1./255
    preprocessing_function=preprocess_input
)

NUM_GPU = 4
batch_size = 100

train_set = train_datagen.flow_from_directory('fashion_mnist_train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

valid_set = test_datagen.flow_from_directory('fashion_mnist_test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

model_txt = 'bi-lstm'
# Helper: Save the model.
savedfilename = os.path.join('checkpoints', 'FashionMNIST_Xception_LRG299_Mul_STD.hdf5')

checkpointer = ModelCheckpoint(savedfilename,
                          monitor='val_acc', verbose=1, 
                          save_best_only=True, mode='max',save_weights_only=True)########

# Helper: TensorBoard
tb = TensorBoard(log_dir=os.path.join('cassava_output', 'logs', model_txt))

# Helper: Save results.
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('cassava_output', 'logs', model_txt + '-' + 'training-' + \
    str(timestamp) + '.log'))

from keras.callbacks import Callback
class EarlyStoppingByAccVal(Callback):
    def __init__(self, monitor='val_acc', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current >= self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping" % epoch)
            self.model.stop_training = True

earlystopping = EarlyStoppingByAccVal(monitor='val_acc', value=0.9900, verbose=1)

#Using multiple models if more than 1 GPU
if NUM_GPU != 1:
    model_mul = multi_gpu_model(model_1, gpus=NUM_GPU)

# print('Loading pretrained weights')
# if NUM_GPU != 1:
#     model_mul.load_weights(savedfilename)
# # else:
# #     model.load_weights(savedfilename)

epochs = 40##!!!
lr = 1e-4
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)

model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

step_size_train=ceil(train_set.n/train_set.batch_size)
step_size_valid=ceil(valid_set.n/valid_set.batch_size)
# step_size_test=ceil(testing_set.n//testing_set.batch_size)

result = model_mul.fit_generator(
    generator = train_set, 
    steps_per_epoch = step_size_train,
    validation_data = valid_set,
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
    callbacks=[earlystopping, checkpointer],
#     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
    verbose=1) 

In [ ]:
import matplotlib.pyplot as plt
history = result
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.grid()
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model_mul.load_weights(os.path.join('checkpoints', 'FashionMNIST_Xception_LRG299_Mul_STD.hdf5'))

In [ ]:
#Non-Groups
#Split training and validation
#Using Expert Data

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras.optimizers import Adam
# from keras.utils import multi_gpu_model

import time, os
from math import ceil
import multiprocessing

savedfilename = os.path.join('checkpoints', 'FashionMNIST_Xception_LRG299_Mul_STD_L2.hdf5')

checkpointer = ModelCheckpoint(savedfilename,
                          monitor='val_acc', verbose=1, 
                          save_best_only=True, mode='max',save_weights_only=True)########

epochs = 15##!!!
lr = 1e-5
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)

model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

step_size_train=ceil(train_set.n/train_set.batch_size)
step_size_valid=ceil(valid_set.n/valid_set.batch_size)
# step_size_test=ceil(testing_set.n//testing_set.batch_size)

result = model_mul.fit_generator(
    generator = train_set, 
    steps_per_epoch = step_size_train,
    validation_data = valid_set,
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
    callbacks=[earlystopping, checkpointer],
#     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
    verbose=1) 

In [ ]:
# #Using multiple models if more than 1 GPU
# NUM_GPU = 4
# if NUM_GPU != 1:
#     model_mul = multi_gpu_model(model_1, gpus=NUM_GPU)

model_mul.load_weights(os.path.join('checkpoints', 'FashionMNIST_Xception_LRG299_Mul_STD_L2.hdf5'))

In [ ]:
#Non-Groups
#Split training and validation
#Using Expert Data

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras.optimizers import Adam
# from keras.utils import multi_gpu_model

import time, os
from math import ceil
import multiprocessing

train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
#     rescale = 1./255
    preprocessing_function=preprocess_input
)

NUM_GPU = 4
batch_size = 100

train_set = train_datagen.flow_from_directory('fashion_mnist_train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

valid_set = test_datagen.flow_from_directory('fashion_mnist_test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

savedfilename = os.path.join('checkpoints', 'FashionMNIST_Xception_LRG299_Mul_STD_L3.hdf5')
checkpointer = ModelCheckpoint(savedfilename,
                          monitor='val_acc', verbose=1, 
                          save_best_only=True, mode='max',save_weights_only=True)########

from keras.callbacks import Callback
class EarlyStoppingByAccVal(Callback):
    def __init__(self, monitor='val_acc', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current >= self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping" % epoch)
            self.model.stop_training = True
earlystopping = EarlyStoppingByAccVal(monitor='val_acc', value=0.9900, verbose=1)

epochs = 15##!!!
lr = 1e-6
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)

model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

step_size_train=ceil(train_set.n/train_set.batch_size)
step_size_valid=ceil(valid_set.n/valid_set.batch_size)
# step_size_test=ceil(testing_set.n//testing_set.batch_size)

result = model_mul.fit_generator(
    generator = train_set, 
    steps_per_epoch = step_size_train,
    validation_data = valid_set,
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
    callbacks=[earlystopping, checkpointer],
#     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
    verbose=1) 

In [ ]:
import numpy as np

print(result.history['val_acc'])
print(np.mean(result.history['val_acc']))
print(np.max(result.history['val_acc']))

print(result.history['acc'])
print(np.mean(result.history['acc']))
print(np.max(result.history['acc']))

In [ ]:
#Using multiple models if more than 1 GPU
NUM_GPU = 4
if NUM_GPU != 1:
    model_mul = multi_gpu_model(model_1, gpus=NUM_GPU)

model_mul.load_weights(os.path.join('checkpoints', 'FashionMNIST_Xception_LRG299_Mul_L3.hdf5'))

In [ ]:
import numpy as np

# PREDICT ON OFFICIAL TEST
test_datagen1 = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

train_set = train_datagen.flow_from_directory('fashion_mnist_train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

test_set1 = test_datagen1.flow_from_directory('fashion_mnist_test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

if NUM_GPU != 1:
    predict1=model_mul.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
# else:
#     predict1=model.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
    
predicted_class_indices=np.argmax(predict1,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions1 = [labels[k] for k in predicted_class_indices]

import pandas as pd

filenames=test_set1.filenames
results=pd.DataFrame({"file_name":filenames,
                      "predicted1":predictions1,
                      })
results.to_csv('FashionMNIST_SeResNet101_MulModels299_2410_01.csv')
results.head()

In [ ]:
mkdir pred_npy

In [ ]:
np.save(os.path.join('pred_npy','FashionMNIST_Xception_LRG299_Mul.npy'), predict1)

In [ ]:
#model 1 - Xception
# load to multi gpu

#use hdf5
NUM_GPU = 4
model_mul = multi_gpu_model(model_1, gpus=NUM_GPU)

hdf5file= os.path.join('checkpoints', 'FashionMNIST_Xception_LRG299_Mul_L3.hdf5')
model_mul.load_weights(hdf5file)

f1_sgl = model_mul.layers[-2]
f1_sgl.save(os.path.join('checkpoints', 'FashionMNIST_Xception_LRG299_Mul_LSTM_L3_SGL.hdf5'))

In [ ]:
model_1.load_weights(os.path.join('checkpoints', 'FashionMNIST_Xception_LRG299_Mul_LSTM_L3_SGL.hdf5'))

In [ ]:
mkdir checkpoints

In [ ]:
#501
#MUL 3 InceptionResNetV2
# from keras.applications import InceptionV3
# from keras.applications import Xception
from keras.applications.inception_resnet_v2 import InceptionResNetV2

from keras.preprocessing import image
from keras.models import Model
from keras.layers import concatenate
from keras.layers import Dense, GlobalAveragePooling2D, Input, Embedding, LSTM
from keras.applications.inception_v3 import preprocess_input

from keras.layers import GaussianNoise

f3_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(299,299,3))  
# for layer in f3_base.layers:
#     layer.trainable = False 
f3_x = f3_base.output
f3_x = GlobalAveragePooling2D()(f3_x)

#Regularization with noise
f3_x = GaussianNoise(0.1)(f3_x)

f3_x = Dense(2048, activation='relu')(f3_x)
f3_x = Dense(10, activation='softmax')(f3_x)
model_3 = Model(inputs=[f3_base.input],outputs=[f3_x])

print(model_3.summary())

In [ ]:
from keras.callbacks import Callback
import pickle
import sys

#Stop training on val_acc
class EarlyStoppingByAccVal(Callback):
    def __init__(self, monitor='val_acc', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current >= self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping" % epoch)
            self.model.stop_training = True

#Save large model using pickle formate instead of h5            
class SaveCheckPoint(Callback):
    def __init__(self, model, dest_folder):
        super(Callback, self).__init__()
        self.model = model
        self.dest_folder = dest_folder
        
        #initiate
        self.best_val_acc = 0
        self.best_val_loss = sys.maxsize #get max value
          
    def on_epoch_end(self, epoch, logs={}):
        val_acc = logs['val_acc']
        val_loss = logs['val_loss']

        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            
            # Save weights in pickle format instead of h5
            print('\nSaving val_acc %f at %s' %(self.best_val_acc, self.dest_folder))
            weigh= self.model.get_weights()

            #now, use pickle to save your model weights, instead of .h5
            #for heavy model architectures, .h5 file is unsupported.
            fpkl= open(self.dest_folder, 'wb') #Python 3
            pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
            fpkl.close()
            
#             model.save('tmp.h5')
        elif val_acc == self.best_val_acc:
            if val_loss < self.best_val_loss:
                self.best_val_loss=val_loss
                
                # Save weights in pickle format instead of h5
                print('\nSaving val_acc %f at %s' %(self.best_val_acc, self.dest_folder))
                weigh= self.model.get_weights()

                #now, use pickle to save your model weights, instead of .h5
                #for heavy model architectures, .h5 file is unsupported.
                fpkl= open(self.dest_folder, 'wb') #Python 3
                pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
                fpkl.close()                    

In [ ]:
#Non-Groups
#Split training and validation
#Using Expert Data

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras.optimizers import Adam
# from keras.utils import multi_gpu_model

import time, os
from math import ceil
import multiprocessing

train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
#     rescale = 1./255
    preprocessing_function=preprocess_input
)

NUM_GPU = 4
batch_size = 120

train_set = train_datagen.flow_from_directory('fashion_mnist_train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

valid_set = test_datagen.flow_from_directory('fashion_mnist_test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

model_txt = 'bi-lstm'
# Helper: Save the model.
savedfilename = os.path.join('checkpoints', 'FashionMNIST_IcpResNetV2_LRG299_Mul_STD.hdf5')

checkpointer = ModelCheckpoint(savedfilename,
                          monitor='val_acc', verbose=1, 
                          save_best_only=True, mode='max',save_weights_only=True)########

# Helper: TensorBoard
tb = TensorBoard(log_dir=os.path.join('cassava_output', 'logs', model_txt))

# Helper: Save results.
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('cassava_output', 'logs', model_txt + '-' + 'training-' + \
    str(timestamp) + '.log'))

earlystopping = EarlyStoppingByAccVal(monitor='val_acc', value=0.9900, verbose=1)

#Using multiple models if more than 1 GPU
if NUM_GPU != 1:
    model_mul = multi_gpu_model(model_3, gpus=NUM_GPU)

# print('Loading pretrained weights')
# if NUM_GPU != 1:
#     model_mul.load_weights(savedfilename)
# # else:
# #     model.load_weights(savedfilename)

epochs = 40##!!!
lr = 1e-4
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)

model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

step_size_train=ceil(train_set.n/train_set.batch_size)
step_size_valid=ceil(valid_set.n/valid_set.batch_size)
# step_size_test=ceil(testing_set.n//testing_set.batch_size)

result = model_mul.fit_generator(
    generator = train_set, 
    steps_per_epoch = step_size_train,
    validation_data = valid_set,
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
    callbacks=[earlystopping, checkpointer],
#     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
    verbose=1) 
  

In [ ]:
#Using multiple models if more than 1 GPU
model_mul.load_weights(os.path.join('checkpoints', 'FashionMNIST_IcpResNetV2_LRG299_Mul_STD.hdf5'))

In [ ]:
# Helper: Save the model.
savedfilename = os.path.join('checkpoints', 'FashionMNIST_IcpResNetV2_LRG299_Mul_STD_L2.hdf5')

checkpointer = ModelCheckpoint(savedfilename,
                          monitor='val_acc', verbose=1, 
                          save_best_only=True, mode='max',save_weights_only=True)########
epochs = 15##!!!
lr = 1e-5
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)

model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

result = model_mul.fit_generator(
    generator = train_set, 
    steps_per_epoch = step_size_train,
    validation_data = valid_set,
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
    callbacks=[earlystopping, checkpointer],
#     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
    verbose=1) 
  

In [ ]:
#Using multiple models if more than 1 GPU
model_mul.load_weights(os.path.join('checkpoints', 'FashionMNIST_IcpResNetV2_LRG299_Mul_L2.hdf5'))

In [ ]:
# Helper: Save the model.
savedfilename = os.path.join('checkpoints', 'FashionMNIST_IcpResNetV2_LRG299_Mul_L3.hdf5')

checkpointer = ModelCheckpoint(savedfilename,
                          monitor='val_acc', verbose=1, 
                          save_best_only=True, mode='max',save_weights_only=True)########
epochs = 5##!!!
lr = 1e-6
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)

model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

result = model_mul.fit_generator(
    generator = train_set, 
    steps_per_epoch = step_size_train,
    validation_data = valid_set,
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
    callbacks=[earlystopping, checkpointer],
#     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
    verbose=1) 
  

In [ ]:
#Using multiple models if more than 1 GPU
import os

NUM_GPU = 4
if NUM_GPU != 1:
    model_mul = multi_gpu_model(model_3, gpus=NUM_GPU)

model_mul.load_weights(os.path.join('checkpoints', 'FashionMNIST_IcpResNetV2_LRG299_Mul_L2.hdf5'))

In [ ]:
#model 1 - InceptionResNetV2
# load to multi gpu

#use hdf5
NUM_GPU = 4
model_mul = multi_gpu_model(model_3, gpus=NUM_GPU)

hdf5file= os.path.join('checkpoints', 'FashionMNIST_IcpResNetV2_LRG299_Mul_L2.hdf5')
model_mul.load_weights(hdf5file)

f1_sgl = model_mul.layers[-2]
f1_sgl.save(os.path.join('checkpoints', 'FashionMNIST_IcpResNetV2_LRG299_Mul_L2_SGL.hdf5'))

In [ ]:
model_3.load_weights(os.path.join('checkpoints', 'FashionMNIST_IcpResNetV2_LRG299_Mul_L2_SGL.hdf5'))

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras.optimizers import Adam
# from keras.utils import multi_gpu_model

import time, os
from math import ceil
import multiprocessing

train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
#     rescale = 1./255
    preprocessing_function=preprocess_input
)

# PREDICT ON OFFICIAL TEST
test_datagen1 = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

batch_size = 120

train_set = train_datagen.flow_from_directory('fashion_mnist_train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

test_set1 = test_datagen1.flow_from_directory('fashion_mnist_test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

if NUM_GPU != 1:
    predict1=model_mul.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
# else:
#     predict1=model.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
    
predicted_class_indices=np.argmax(predict1,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions1 = [labels[k] for k in predicted_class_indices]

import pandas as pd

filenames=test_set1.filenames
results=pd.DataFrame({"file_name":filenames,
                      "predicted1":predictions1,
                      })
results.to_csv('FashionMNIST_IcpResNetV2_MulModels299_2510_01.csv')
results.head()

In [ ]:
np.save(os.path.join('pred_npy','FashionMNIST_IcpResNetV2_LRG299_Mul.npy'), predict1)

In [ ]:
mean_pred3_1 = np.load(os.path.join('pred_npy','FashionMNIST_SEResNext101_LRG299_Mul.npy'))
mean_pred3_2 = np.load(os.path.join('pred_npy','FashionMNIST_Xception_LRG299_Mul.npy'))
mean_pred3_3 = np.load(os.path.join('pred_npy','FashionMNIST_IcpResNetV2_LRG299_Mul.npy'))
mean_pred3 = (mean_pred3_1+mean_pred3_2+mean_pred3_3)/3
print(mean_pred3[0:9])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from math import ceil
import numpy as np

batch_size = 20

#Crop-Official Test
def random_crop(img, random_crop_size):
    # Note: image_data_format is 'channel_last'
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]

def crop_generator(batches, crop_length):
    """Generate random crops from the image batches"""
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)

train_datagen = ImageDataGenerator(

)

train_set = train_datagen.flow_from_directory('fashion_mnist_train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )
        
test_datagen_crop = ImageDataGenerator(
)

testing_set_crop = test_datagen_crop.flow_from_directory('fashion_mnist_test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="training"
                                              )
#customized generator
test_crops = crop_generator(testing_set_crop, 299)

step_size_test_crop = ceil(testing_set_crop.n/testing_set_crop.batch_size)

predicted_class_indices_mean=np.argmax(mean_pred3,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
finalpre = [labels[k] for k in predicted_class_indices_mean]

import pandas as pd
filenames=testing_set_crop.filenames
results=pd.DataFrame({"id":filenames,
                      "predicted":finalpre,
                      })
results.to_csv('FashionMNIST_MulModels299_AVG_Assembe_V2_2510_2.csv')
results.head(20)

In [ ]:
cp FashionMNIST_MulModels299_AVG_Assembe_V2_2510_2.csv /home/bribeiro/Phong/Nat19/FashionMNIST_MulModels299_AVG_Assembe_V2_2510_2.csv

In [ ]:
# model_1.load_weights(os.path.join('checkpoints', 'Cassava_NonGrp_S1_Xception_LRG501_Mul_Pretrained_STD_SGL.hdf5'))
# # model_2.load_weights(os.path.join('checkpoints', 'Cassava_NonGrp_S1_Inception_LRG501_Mul_Pretrained_STD_SGL.hdf5'))
# model_3.load_weights(os.path.join('checkpoints', 'Cassava_NonGrp_S1_IcpResNetV2_LRG501_Mul_Pretrained_STD_SGL.hdf5'))

In [ ]:
# 3 models
# 501
# pre-trainned weight
# get pretrained [-1] layer

from keras.applications import InceptionV3
from keras.applications import Xception
from keras.applications.inception_resnet_v2 import InceptionResNetV2

from keras.preprocessing import image
from keras.models import Model
from keras.layers import concatenate
from keras.layers import Dense, GlobalAveragePooling2D, Input, Embedding, LSTM, LeakyReLU, Dropout
#from keras.applications.inception_v3 import preprocess_input
from keras_applications.imagenet_utils import preprocess_input

f1_mul_x = model_1.layers[-1].output 
f2_mul_x = model_2.layers[-1].output
f3_mul_x = model_3.layers[-1].output

x = concatenate([f1_mul_x, f2_mul_x, f3_mul_x])

x = Dense(4096)(x)
x = LeakyReLU(0.2)(x)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)
model_mul = Model(inputs=[model_1.get_input_at(0),model_2.get_input_at(0),model_3.get_input_at(0)],outputs=[x])

print(model_mul.summary())

In [ ]:
for layer in model_mul.layers[:-4]:
    layer.trainable=False
# for layer in model.layers[87:]:
#     layer.trainable=True)
for i,layer in enumerate(model_mul.layers):
    print(i,layer.name,layer.trainable)

In [ ]:
# generators for three inputs
def train_generator_three_img(X1, batch_size):
    gen = ImageDataGenerator(    
        rotation_range=30,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.3,
        horizontal_flip=True,
        vertical_flip=True,##
    #     brightness_range=[0.5, 1.5],##
        channel_shift_range=10,##
        fill_mode='nearest')
    
    genX1 = gen.flow_from_directory(X1, batch_size=batch_size, seed=1, target_size = (299, 299))    
    while True:
        X1i = genX1.next()
        yield [X1i[0], X1i[0], X1i[0]], X1i[1]
        
def valid_generator_three_img(X1, batch_size):
    gen = ImageDataGenerator()

    genX1 = gen.flow_from_directory(X1, batch_size=batch_size, seed=1, target_size = (299, 299))        
    while True:
        X1i = genX1.next()
        yield [X1i[0], X1i[0], X1i[0]], X1i[1]    
    
def test_generator_three_img(X1, batch_size):
    gen = ImageDataGenerator(
#         preprocessing_function=preprocess_input
    )

    genX1 = gen.flow_from_directory(X1, batch_size=batch_size, shuffle=False, target_size = (299, 299))        
    while True:
        X1i_0, X1i_1 = next(genX1)
        yield [X1i_0, X1i_0, X1i_0]            

In [ ]:
# #Non-Groups
# #Split training and validation
# #Using Expert Data

# from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
# from keras.optimizers import Adam
# # from keras.utils import multi_gpu_model

# import time, os
# from math import ceil
# import multiprocessing

# ## fix for multi_gpu_model prediction time longer
# from keras.layers import Lambda, concatenate
# from keras import Model
# import tensorflow as tf

# def multi_gpu_model(model, gpus):
#     if isinstance(gpus, (list, tuple)):
#         num_gpus = len(gpus)
#         target_gpu_ids = gpus
#     else:
#         num_gpus = gpus
#         target_gpu_ids = range(num_gpus)

#     def get_slice(data, i, parts):
#         shape = tf.shape(data)
#         batch_size = shape[:1]
#         input_shape = shape[1:]
#         step = batch_size // parts
#         if i == num_gpus - 1:
#             size = batch_size - step * i
#         else:
#             size = step
#         size = tf.concat([size, input_shape], axis=0)
#         stride = tf.concat([step, input_shape * 0], axis=0)
#         start = stride * i
#         return tf.slice(data, start, size)

#     all_outputs = []
#     for i in range(len(model.outputs)):
#         all_outputs.append([])

#     # Place a copy of the model on each GPU,
#     # each getting a slice of the inputs.
#     for i, gpu_id in enumerate(target_gpu_ids):
#         with tf.device('/gpu:%d' % gpu_id):
#             with tf.name_scope('replica_%d' % gpu_id):
#                 inputs = []
#                 # Retrieve a slice of the input.
#                 for x in model.inputs:
#                     input_shape = tuple(x.get_shape().as_list())[1:]
#                     slice_i = Lambda(get_slice,
#                                    output_shape=input_shape,
#                                    arguments={'i': i,
#                                               'parts': num_gpus})(x)
#                     inputs.append(slice_i)

#                 # Apply model on slice
#                 # (creating a model replica on the target device).
#                 outputs = model(inputs)
#                 if not isinstance(outputs, list):
#                     outputs = [outputs]

#                 # Save the outputs for merging back together later.
#                 for o in range(len(outputs)):
#                     all_outputs[o].append(outputs[o])

#     # Merge outputs on CPU.
#     with tf.device('/cpu:0'):
#         merged = []
#         for name, outputs in zip(model.output_names, all_outputs):
#             merged.append(concatenate(outputs,
#                                     axis=0, name=name))
#         return Model(model.inputs, merged)

# train_datagen = ImageDataGenerator(
# #     rescale = 1./255,
#     rotation_range=30,
#     width_shift_range=0.3,
#     height_shift_range=0.3,
#     shear_range=0.3,
#     zoom_range=0.3,
#     horizontal_flip=True,
#     vertical_flip=True,##
# #     brightness_range=[0.5, 1.5],##
#     channel_shift_range=10,##
#     fill_mode='nearest',
#     preprocessing_function=preprocess_input,
# )

# test_datagen = ImageDataGenerator(
# #     rescale = 1./255
#     preprocessing_function=preprocess_input
# )

# NUM_GPU = 4
# batch_size = 120

# training_set = train_datagen.flow_from_directory('fashion_mnist_train_resized_299',
#                                                  target_size = (299, 299),
#                                                  batch_size = batch_size,
#                                                  class_mode = 'categorical',
#                                                  shuffle=True,
#                                                  seed=7,
# #                                                  subset="training"
#                                               )

# valid_set = test_datagen.flow_from_directory('fashion_mnist_test_resized_299',
#                                                  target_size = (299, 299),
#                                                  batch_size = batch_size,
#                                                  class_mode = 'categorical',
#                                                  shuffle=False,
#                                                  seed=7,
# #                                                  subset="validation"
#                                              )

# model_txt = 'bi-lstm'
# # Helper: Save the model.
# savedfilename = os.path.join('checkpoints', 'FashionMNIST_LRG299_Mul_Pretrained_SeNet_Xcep_IcpRes_STD.pkl')

# checkpointer = ModelCheckpoint(savedfilename,
#                           monitor='val_acc', verbose=1, 
#                           save_best_only=True, mode='max',save_weights_only=False)

# # Helper: TensorBoard
# tb = TensorBoard(log_dir=os.path.join('fmnist_output', 'logs', model_txt))

# # Helper: Save results.
# timestamp = time.time()
# csv_logger = CSVLogger(os.path.join('fmnist_output', 'logs', model_txt + '-' + 'training-' + \
#     str(timestamp) + '.log'))

# from keras.callbacks import Callback
# class EarlyStoppingByAccVal(Callback):
#     def __init__(self, monitor='val_acc', value=0.00001, verbose=0):
#         super(Callback, self).__init__()
#         self.monitor = monitor
#         self.value = value
#         self.verbose = verbose

#     def on_epoch_end(self, epoch, logs={}):
#         current = logs.get(self.monitor)
#         if current is None:
#             warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

#         if current >= self.value:
#             if self.verbose > 0:
#                 print("Epoch %05d: early stopping" % epoch)
#             self.model.stop_training = True

# earlystopping = EarlyStoppingByAccVal(monitor='val_acc', value=0.99, verbose=1)

# #Using multiple models if more than 1 GPU
# if NUM_GPU != 1:
#     model_mul = multi_gpu_model(model_mul, gpus=NUM_GPU)

# # print('Loading pretrained weights')
# # if NUM_GPU != 1:
# #     model_mul.load_weights(savedfilename)
# # else:
# #     model.load_weights(savedfilename)

# epochs = 30##!!!
# lr = 1e-5
# decay = lr/epochs
# optimizer = Adam(lr=lr, decay=decay)

# model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

# step_size_train=ceil(training_set.n/training_set.batch_size)
# step_size_valid=ceil(valid_set.n/valid_set.batch_size)
# # step_size_test=ceil(testing_set.n//testing_set.batch_size)

# result = model_mul.fit_generator(
#     generator = train_generator_three_img('fashion_mnist_train_resized_299',batch_size=batch_size), 
#     steps_per_epoch = step_size_train,
#     validation_data = valid_generator_three_img('fashion_mnist_test_resized_299',batch_size=batch_size),
#     validation_steps = step_size_valid,
#     shuffle=True,
#     epochs=epochs,
# #     callbacks=[tb, csv_logger, checkpointer, earlystopping], 
#     callbacks=[checkpointer, earlystopping], 
#     verbose=1)    

In [ ]:
#Non-Groups
#Split training and validation
#Using Expert Data

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras.optimizers import Adam
# from keras.utils import multi_gpu_model

import time, os
from math import ceil
import multiprocessing

## fix for multi_gpu_model prediction time longer
from keras.layers import Lambda, concatenate
from keras import Model
import tensorflow as tf

NUM_GPU = 4
batch_size = 120
#Using multiple models if more than 1 GPU
if NUM_GPU != 1:
    model_mul = multi_gpu_model(model_mul, gpus=NUM_GPU)

epochs = 30##!!!
lr = 1e-5
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)

if NUM_GPU != 1:
    model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
#     rescale = 1./255
    preprocessing_function=preprocess_input
)

train_set = train_datagen.flow_from_directory('fashion_mnist_train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

valid_set = test_datagen.flow_from_directory('fashion_mnist_test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

model_txt = 'bi-lstm'
# Helper: Save the model.
savedfilename = os.path.join('checkpoints', 'FashionMNIST_LRG299_Mul_Pretrained_SeNet_Xcep_IcpRes_STD.pkl')

earlystopping = EarlyStoppingByAccVal(monitor='val_acc', value=0.99, verbose=1)
savecheckpoint = SaveCheckPoint(model_2, savedfilename)

# Helper: TensorBoard
tb = TensorBoard(log_dir=os.path.join('fmnist_output', 'logs', model_txt))

# Helper: Save results.
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('fmnist_output', 'logs', model_txt + '-' + 'training-' + \
    str(timestamp) + '.log'))

step_size_train=ceil(training_set.n/training_set.batch_size)
step_size_valid=ceil(valid_set.n/valid_set.batch_size)
# step_size_test=ceil(testing_set.n//testing_set.batch_size)

result = model_mul.fit_generator(
    generator = train_generator_three_img('fashion_mnist_train_resized_299',batch_size=batch_size), 
    steps_per_epoch = step_size_train,
    validation_data = valid_generator_three_img('fashion_mnist_test_resized_299',batch_size=batch_size),
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
#     callbacks=[tb, csv_logger, savecheckpoint, earlystopping], 
    callbacks=[savecheckpoint, earlystopping], 
    verbose=1)    

In [ ]:
# Visualize training history
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy

# list all data in history
# history = result.history
#print(history.history.keys())
# summarize history for accuracy
plt.plot(result.history['acc'])
plt.plot(result.history['val_acc'])
plt.title('Mul-ResNext - 501 - Noise model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(result.history['loss'])
plt.plot(result.history['val_loss'])
plt.title('Mul-ResNext - 501 - Noise model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# 3 models
# 501
# pre-trainned weight
# get pretrained [-1] layer

from keras.applications import InceptionV3
from keras.applications import Xception
from keras.applications.inception_resnet_v2 import InceptionResNetV2

from keras.preprocessing import image
from keras.models import Model
from keras.layers import concatenate
from keras.layers import Dense, GlobalAveragePooling2D, Input, Embedding, LSTM, LeakyReLU, Dropout
#from keras.applications.inception_v3 import preprocess_input
from keras_applications.imagenet_utils import preprocess_input

f1_mul_x = model_1.layers[-1].output 
f2_mul_x = model_2.layers[-1].output
f3_mul_x = model_3.layers[-1].output

x = concatenate([f1_mul_x, f2_mul_x, f3_mul_x])

x = Reshape([1,30])(x)  
x = Bidirectional(LSTM(1024, 
                                 return_sequences=False, 
#                                  dropout=0.8
                                ),
                            input_shape=[1,30],
                            merge_mode='concat')(x)

# x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)
model_mul = Model(inputs=[model_1.get_input_at(0),model_2.get_input_at(0),model_3.get_input_at(0)],outputs=[x])

print(model_mul.summary())

In [ ]:
#Non-Groups
#Split training and validation
#Using Expert Data

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras.optimizers import Adam
# from keras.utils import multi_gpu_model

import time, os
from math import ceil
import multiprocessing

## fix for multi_gpu_model prediction time longer
from keras.layers import Lambda, concatenate
from keras import Model
import tensorflow as tf

train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
#     rescale = 1./255
    preprocessing_function=preprocess_input
)

NUM_GPU = 4
batch_size = 120

training_set = train_datagen.flow_from_directory('fashion_mnist_train_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

valid_set = test_datagen.flow_from_directory('fashion_mnist_test_resized_299',
                                                 target_size = (299, 299),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

model_txt = 'bi-lstm'
# Helper: Save the model.
savedfilename = os.path.join('checkpoints', 'FashionMNIST_LRG299_Mul_Pretrained_SeNet_Xcep_IcpRes_BiLSTM.hdf5')

checkpointer = ModelCheckpoint(savedfilename,
                          monitor='val_acc', verbose=1, 
                          save_best_only=True, mode='max',save_weights_only=False)

# Helper: TensorBoard
tb = TensorBoard(log_dir=os.path.join('fmnist_output', 'logs', model_txt))

# Helper: Save results.
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('fmnist_output', 'logs', model_txt + '-' + 'training-' + \
    str(timestamp) + '.log'))

from keras.callbacks import Callback
class EarlyStoppingByAccVal(Callback):
    def __init__(self, monitor='val_acc', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current >= self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping" % epoch)
            self.model.stop_training = True

earlystopping = EarlyStoppingByAccVal(monitor='val_acc', value=0.99, verbose=1)

#Using multiple models if more than 1 GPU
if NUM_GPU != 1:
    model_mul = multi_gpu_model(model_mul, gpus=NUM_GPU)

# print('Loading pretrained weights')
# if NUM_GPU != 1:
#     model_mul.load_weights(savedfilename)
# else:
#     model.load_weights(savedfilename)

epochs = 20##!!!
lr = 1e-4
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)

model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

step_size_train=ceil(training_set.n/training_set.batch_size)
step_size_valid=ceil(valid_set.n/valid_set.batch_size)
# step_size_test=ceil(testing_set.n//testing_set.batch_size)

result = model_mul.fit_generator(
    generator = train_generator_three_img('fashion_mnist_train_resized_299',batch_size=batch_size), 
    steps_per_epoch = step_size_train,
    validation_data = valid_generator_three_img('fashion_mnist_test_resized_299',batch_size=batch_size),
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
#     callbacks=[tb, csv_logger, checkpointer, earlystopping], 
    callbacks=[tb, csv_logger, earlystopping], 
    verbose=1)    

In [ ]:
#change test_generator
import numpy as np

# PREDICT ON OFFICIAL TEST
test_datagen1 = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

train_set = train_datagen.flow_from_directory('train',
                                                 target_size = (501, 501),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

test_set1 = test_datagen1.flow_from_directory('test',
                                                 target_size = (501, 501),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

if NUM_GPU != 1:
    predict1=model_mul.predict_generator(test_generator_three_img('test',batch_size), steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
# else:
#     predict1=model.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
    
predicted_class_indices=np.argmax(predict1,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions1 = [labels[k] for k in predicted_class_indices]

import pandas as pd

filenames=test_set1.filenames
results=pd.DataFrame({"file_name":filenames,
                      "predicted1":predictions1,
                      })
results.to_csv('Cassava_NonGrp_MulModels501_2208_v1.csv')
results.head(10)

In [ ]:
cp Cassava_NonGrp_MulModels501_2208_v1.csv /home/bribeiro/Phong/Nat19/Cassava_NonGrp_MulModels501_2208_v1.csv

In [ ]:
#MULTI GPUS

from keras.preprocessing.image import ImageDataGenerator
from math import ceil
import numpy as np

#Crop-Official Test
def random_crop(img, random_crop_size):
    # Note: image_data_format is 'channel_last'
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]

def crop_generator_three_models(batches, crop_length):
    """Generate random crops from the image batches"""
     
    while True:
        batch_x, _ = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
        yield [batch_crops, batch_crops, batch_crops]

gen = ImageDataGenerator(
#     rescale = 1./255,
#         preprocessing_function=preprocess_input
)

genX1 = gen.flow_from_directory('test',
                                 target_size = (541, 541),
                                 batch_size = batch_size,
                                 class_mode = 'categorical',
                                 shuffle=False,
                                 seed=7,
                              )

#steps
step_size_test_crop = ceil(genX1.n/genX1.batch_size)

#customized generator
test_crops = crop_generator_three_models(genX1, 501)

#number of crops
tta_steps = 7
# predictions = []

for i in range(tta_steps):
    print('iteration = %d' %i)
    #reset index
    genX1.reset()
    
    preds=model_mul.predict_generator(test_crops,steps = step_size_test_crop,verbose=1)  
    predictions.append(preds)

mean_pred = np.mean(predictions, axis=0)

predicted_class_indices_mean=np.argmax(mean_pred,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
finalpre = [labels[k] for k in predicted_class_indices_mean]

import pandas as pd
filenames=testing_set_crop.filenames
results=pd.DataFrame({"id":filenames,
                      "predicted":finalpre,
                      })
results.to_csv('Cassava_NonGrp_MulModels501_2208_v5.csv')
results.head(11)

In [ ]:
cp Cassava_NonGrp_MulModels501_2208_v5.csv /home/bribeiro/Phong/Nat19/Cassava_NonGrp_MulModels501_2208_v5.csv

In [ ]:
import pickle

#use get_weights() command to get your model weights
weigh= model_mul.get_weights()

#now, use pickle to save your model weights, instead of .h5
#for heavy model architectures, .h5 file is unsupported.
pklfile= os.path.join('checkpoints', 'Cassava_NonGrp_S1_LRG501_Mul_Pretrained_STD_09076.pkl')

fpkl= open(pklfile, 'wb') #Python 3
pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
fpkl.close()

In [ ]:
#use pickle to load model weights

pklfile= os.path.join('checkpoints', 'Cassava_NonGrp_S1_LRG501_Mul_Pretrained_STD_09076.pkl')

f= open(pklfile, 'rb')     #Python 3                 
weigh= pickle.load(f);                
f.close();

gpus_model_mul = multi_gpu_model(model_mul, gpus=NUM_GPU)
#use set_weights to load the modelweights into the model architecture
gpus_model_mul.set_weights(weigh)

In [ ]:
epochs = 50##!!!

result = model_mul.fit_generator(
    generator = train_generator_three_img('train',batch_size=batch_size), 
    steps_per_epoch = step_size_train,
    validation_data = valid_generator_three_img('valid',batch_size=batch_size),
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
#     callbacks=[tb, csv_logger, checkpointer, earlystopping], 
    callbacks=[tb, csv_logger, earlystopping], 
    verbose=1)    

In [ ]:
# Visualize training history
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy

# list all data in history
# history = result.history
#print(history.history.keys())
# summarize history for accuracy
plt.plot(result.history['acc'])
plt.plot(result.history['val_acc'])
plt.title('Mul-ResNext - 501 - Noise model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(result.history['loss'])
plt.plot(result.history['val_loss'])
plt.title('Mul-ResNext - 501 - Noise model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#change test_generator
import numpy as np

# PREDICT ON OFFICIAL TEST
test_datagen1 = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

train_set = train_datagen.flow_from_directory('train',
                                                 target_size = (501, 501),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

test_set1 = test_datagen1.flow_from_directory('test',
                                                 target_size = (501, 501),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

if NUM_GPU != 1:
    predict1=model_mul.predict_generator(test_generator_three_img('test',batch_size), steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
# else:
#     predict1=model.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
    
predicted_class_indices=np.argmax(predict1,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions1 = [labels[k] for k in predicted_class_indices]

import pandas as pd

filenames=test_set1.filenames
results=pd.DataFrame({"file_name":filenames,
                      "predicted1":predictions1,
                      })
results.to_csv('Cassava_NonGrp_MulModels501_2208_v6.csv')
results.head(10)

In [ ]:
cp Cassava_NonGrp_MulModels501_2208_v6.csv /home/bribeiro/Phong/Nat19/Cassava_NonGrp_MulModels501_2208_v6.csv

In [ ]:
epochs = 10##!!!

lr = 1e-5
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)
model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

earlystopping = EarlyStoppingByAccVal(monitor='val_acc', value=0.91, verbose=1)

result = model_mul.fit_generator(
    generator = train_generator_three_img('train',batch_size=batch_size), 
    steps_per_epoch = step_size_train,
    validation_data = valid_generator_three_img('valid',batch_size=batch_size),
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
#     callbacks=[tb, csv_logger, checkpointer, earlystopping], 
    callbacks=[tb, csv_logger, earlystopping], 
    verbose=1) 

In [ ]:
#train after reload weight (val_acc: 0.9076)

epochs = 40##!!!

lr = 1e-5
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)
gpus_model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

earlystopping = EarlyStoppingByAccVal(monitor='val_acc', value=0.913, verbose=1)

result = gpus_model_mul.fit_generator(
    generator = train_generator_three_img('train',batch_size=batch_size), 
    steps_per_epoch = step_size_train,
    validation_data = valid_generator_three_img('valid',batch_size=batch_size),
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
#     callbacks=[tb, csv_logger, checkpointer, earlystopping], 
    callbacks=[tb, csv_logger, earlystopping], 
    verbose=1) 

In [ ]:
#change test_generator
import numpy as np

# PREDICT ON OFFICIAL TEST
test_datagen1 = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

train_set = train_datagen.flow_from_directory('train',
                                                 target_size = (501, 501),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

test_set1 = test_datagen1.flow_from_directory('test',
                                                 target_size = (501, 501),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

if NUM_GPU != 1:
    predict1=gpus_model_mul.predict_generator(test_generator_three_img('test',batch_size), steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
# else:
#     predict1=model.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
    
predicted_class_indices=np.argmax(predict1,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions1 = [labels[k] for k in predicted_class_indices]

import pandas as pd

filenames=test_set1.filenames
results=pd.DataFrame({"file_name":filenames,
                      "predicted1":predictions1,
                      })
results.to_csv('Cassava_NonGrp_MulModels501_2208_v8.csv')
results.head(10)

In [ ]:
cp Cassava_NonGrp_MulModels501_2208_v8.csv /home/bribeiro/Phong/Nat19/Cassava_NonGrp_MulModels501_2208_v8.csv

In [ ]:
#MULTI GPUS

from keras.preprocessing.image import ImageDataGenerator
from math import ceil
import numpy as np

#Crop-Official Test
def random_crop(img, random_crop_size):
    # Note: image_data_format is 'channel_last'
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]

def crop_generator_three_models(batches, crop_length):
    """Generate random crops from the image batches"""
     
    while True:
        batch_x, _ = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
        yield [batch_crops, batch_crops, batch_crops]

gen = ImageDataGenerator(
#     rescale = 1./255,
#         preprocessing_function=preprocess_input
)

genX1 = gen.flow_from_directory('test',
                                 target_size = (541, 541),
                                 batch_size = batch_size,
                                 class_mode = 'categorical',
                                 shuffle=False,
                                 seed=7,
                              )

#steps
step_size_test_crop = ceil(genX1.n/genX1.batch_size)

#customized generator
test_crops = crop_generator_three_models(genX1, 501)

#number of crops
tta_steps = 5
predictions = []

for i in range(tta_steps):
    print('iteration = %d / %d' %(i+1,tta_steps))
    #reset index
    genX1.reset()
    
    preds=gpus_model_mul.predict_generator(test_crops,steps = step_size_test_crop,verbose=1)  
    predictions.append(preds)

mean_pred = np.mean(predictions, axis=0)

predicted_class_indices_mean=np.argmax(mean_pred,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
finalpre = [labels[k] for k in predicted_class_indices_mean]

import pandas as pd
filenames=testing_set_crop.filenames
results=pd.DataFrame({"id":filenames,
                      "predicted":finalpre,
                      })
results.to_csv('Cassava_NonGrp_MulModels501_2208_v10.csv')
results.head(11)

In [ ]:
cp Cassava_NonGrp_MulModels501_2208_v10.csv /home/bribeiro/Phong/Nat19/Cassava_NonGrp_MulModels501_2208_v11.csv

In [ ]:
#train after reload weight (val_acc: 0.9076)

epochs = 40##!!!

lr = 1e-5
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)
gpus_model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

earlystopping = EarlyStoppingByAccVal(monitor='val_acc', value=0.919, verbose=1)

result = gpus_model_mul.fit_generator(
    generator = train_generator_three_img('train',batch_size=batch_size), 
    steps_per_epoch = step_size_train,
    validation_data = valid_generator_three_img('valid',batch_size=batch_size),
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
#     callbacks=[tb, csv_logger, checkpointer, earlystopping], 
    callbacks=[tb, csv_logger, earlystopping], 
    verbose=1) 

In [ ]:
# Visualize training history
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy

# list all data in history
# history = result.history
#print(history.history.keys())
# summarize history for accuracy
plt.plot(result.history['acc'])
plt.plot(result.history['val_acc'])
plt.title('Mul-ResNext - 501 - Noise model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(result.history['loss'])
plt.plot(result.history['val_loss'])
plt.title('Mul-ResNext - 501 - Noise model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#change test_generator
import numpy as np

# PREDICT ON OFFICIAL TEST
test_datagen1 = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

train_set = train_datagen.flow_from_directory('train',
                                                 target_size = (501, 501),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

test_set1 = test_datagen1.flow_from_directory('test',
                                                 target_size = (501, 501),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

if NUM_GPU != 1:
    predict1=gpus_model_mul.predict_generator(test_generator_three_img('test',batch_size), steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
# else:
#     predict1=model.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
    
predicted_class_indices=np.argmax(predict1,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions1 = [labels[k] for k in predicted_class_indices]

import pandas as pd

filenames=test_set1.filenames
results=pd.DataFrame({"file_name":filenames,
                      "predicted1":predictions1,
                      })
results.to_csv('Cassava_NonGrp_MulModels501_2208_v12.csv')
results.head(10)

In [ ]:
cp Cassava_NonGrp_MulModels501_2208_v12.csv /home/bribeiro/Phong/Nat19/Cassava_NonGrp_MulModels501_2208_v12.csv

In [ ]:
#train after reload weight (val_acc: 0.9076)

epochs = 40##!!!

# lr = 1e-5
# decay = lr/epochs
lr = 1e-6
decay = 0
optimizer = Adam(lr=lr, decay=decay)
gpus_model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

earlystopping = EarlyStoppingByAccVal(monitor='val_acc', value=0.925, verbose=1)

result = gpus_model_mul.fit_generator(
    generator = train_generator_three_img('train',batch_size=batch_size), 
    steps_per_epoch = step_size_train,
    validation_data = valid_generator_three_img('valid',batch_size=batch_size),
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
#     callbacks=[tb, csv_logger, checkpointer, earlystopping], 
    callbacks=[tb, csv_logger, earlystopping], 
    verbose=1) 

In [ ]:
#seresnext50
#Non-Groups
#Split training and validation

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras.optimizers import Adam
# from keras.utils import multi_gpu_model

import time, os
from math import ceil
import multiprocessing

train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
#     rescale = 1./255
    preprocessing_function=preprocess_input
)

NUM_GPU = 4
batch_size = 20

train_set = train_datagen.flow_from_directory('train',
                                                 target_size = (501, 501),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

valid_set = test_datagen.flow_from_directory('valid',
                                                 target_size = (501, 501),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

model_txt = 'bi-lstm'
# Helper: Save the model.
savedfilename = os.path.join('checkpoints', 'Cassava_NonGrp_S1_SEResNext50_LRG501_Mul_Pretrained_STD_L4.hdf5')

checkpointer = ModelCheckpoint(savedfilename,
                          monitor='val_acc', verbose=1, 
                          save_best_only=True, mode='max',save_weights_only=True)########

# Helper: TensorBoard
tb = TensorBoard(log_dir=os.path.join('cassava_output', 'logs', model_txt))

# Helper: Save results.
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('cassava_output', 'logs', model_txt + '-' + 'training-' + \
    str(timestamp) + '.log'))

from keras.callbacks import Callback
class EarlyStoppingByAccVal(Callback):
    def __init__(self, monitor='val_acc', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current >= self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping" % epoch)
            self.model.stop_training = True

earlystopping = EarlyStoppingByAccVal(monitor='val_acc', value=0.8870, verbose=1)##!!!!!!

#Using multiple models if more than 1 GPU
if NUM_GPU != 1:
    model_mul = multi_gpu_model(model_2, gpus=NUM_GPU)

# print('Loading pretrained weights')
# if NUM_GPU != 1:
#     model_mul.load_weights(savedfilename)
# # else:
# #     model.load_weights(savedfilename)

epochs = 30##!!!
lr = 1e-5
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)

model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

step_size_train=ceil(train_set.n/train_set.batch_size)
step_size_valid=ceil(valid_set.n/valid_set.batch_size)
# step_size_test=ceil(testing_set.n//testing_set.batch_size)

result = model_mul.fit_generator(
    generator = train_set, 
    steps_per_epoch = step_size_train,
    validation_data = valid_set,
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
#     callbacks=[tb, csv_logger, checkpointer, earlystopping], 
    callbacks=[tb, csv_logger, earlystopping],
    verbose=1) 

In [ ]:
#0.8898756673450369
#0.8827708640911651
max(result.history['val_acc'])

In [ ]:
# Visualize training history
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy

# list all data in history
# history = result.history
#print(history.history.keys())
# summarize history for accuracy
plt.plot(result.history['acc'])
plt.plot(result.history['val_acc'])
plt.title('SeNet50 - 501 - Noise model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(result.history['loss'])
plt.plot(result.history['val_loss'])
plt.title('SeNet50 - 501 - Noise model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model_mul.save_weights(savedfilename)

In [ ]:
model_mul.save_weights(savedfilename)
f2_sgl = model_mul.layers[-2]
f2_sgl.save(os.path.join('checkpoints', 'Cassava_NonGrp_S1_SEResNext50_LRG450_Mul_Pretrained_STD_L3_SGL.hdf5'))

In [ ]:
import numpy as np

# PREDICT ON OFFICIAL TEST
test_datagen1 = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

train_set = train_datagen.flow_from_directory('train',
                                                 target_size = (450, 450),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

test_set1 = test_datagen1.flow_from_directory('test',
                                                 target_size = (450, 450),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

if NUM_GPU != 1:
    predict1=model_mul.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
# else:
#     predict1=model.predict_generator(test_set1, steps = ceil(test_set1.n/test_set1.batch_size),verbose=1)
    
predicted_class_indices=np.argmax(predict1,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions1 = [labels[k] for k in predicted_class_indices]

import pandas as pd

filenames=test_set1.filenames
results=pd.DataFrame({"file_name":filenames,
                      "predicted1":predictions1,
                      })
results.to_csv('Cassava_NonGrp_MulModels450_2108_v1.csv')
results.head()

In [ ]:
cp Cassava_NonGrp_MulModels450_2108_v1.csv /home/bribeiro/Phong/Nat19/Cassava_NonGrp_MulModels450_2108_v1.csv

In [ ]:
model_2.load_weights(os.path.join('checkpoints', 'Cassava_NonGrp_S1_SEResNext50_LRG450_Mul_Pretrained_STD_L3_SGL.hdf5'))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from math import ceil
import numpy as np

batch_size = 20

#Crop-Official Test
def random_crop(img, random_crop_size):
    # Note: image_data_format is 'channel_last'
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]

def crop_generator(batches, crop_length):
    """Generate random crops from the image batches"""
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)

test_datagen_crop = ImageDataGenerator(
#     rescale = 1./255,
    preprocessing_function=preprocess_input
)

testing_set_crop = test_datagen_crop.flow_from_directory('test',
                                                 target_size = (500, 500),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="training"
                                              )
#customized generator
test_crops = crop_generator(testing_set_crop, 450)

step_size_test_crop = ceil(testing_set_crop.n/testing_set_crop.batch_size)

tta_steps = 3
predictions = []

# import tensorflow as tf
# with tf.device('/gpu:0'):
for i in range(tta_steps):
    print(i)
    testing_set_crop.reset()
#     if NUM_GPU != 1:
#         preds=mul_model.predict_generator(test_crops, 
#                                            steps = step_size_test_crop,
#                                            max_queue_size=16,
# #                                                use_multiprocessing=True,
#                                            workers=1,
#                                            verbose=1)    
#     else:
#         preds=model.predict_generator(test_crops, 
#                                            steps = step_size_test_crop,
#                                            max_queue_size=16,
# #                                                use_multiprocessing=True,
#                                            workers=1,
#                                            verbose=1)  
    preds=model_2.predict_generator(test_crops,steps = step_size_test_crop,verbose=1)  
    predictions.append(preds)

mean_pred = np.mean(predictions, axis=0)

predicted_class_indices_mean=np.argmax(mean_pred,axis=1)
labels = (train_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
finalpre = [labels[k] for k in predicted_class_indices_mean]

import pandas as pd
filenames=testing_set_crop.filenames
results=pd.DataFrame({"id":filenames,
                      "predicted":finalpre,
                      })
results.to_csv('Cassava_NonGrp_MulModels450_2108_v3.csv')
results.head(10)

In [ ]:
cp Cassava_NonGrp_MulModels450_2108_v3.csv /home/bribeiro/Phong/Nat19/Cassava_NonGrp_MulModels450_2108_v3.csv

In [ ]:
result = model_mul.fit_generator(
    generator = train_set, 
    steps_per_epoch = step_size_train,
    validation_data = valid_set,
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
    callbacks=[tb, csv_logger, checkpointer, earlystopping],        
    verbose=1) 

In [ ]:
from classification_models.keras import Classifiers
from keras.models import Model
# from keras.layers import concatenate
from keras.layers import Dense, GlobalAveragePooling2D, Input, Embedding, LSTM
# from keras.applications.inception_v3 import preprocess_input

from keras.layers import GaussianNoise

nasnet, preprocess_input = Classifiers.get('senet154')

f4_base = nasnet(input_shape=(501,501,3), weights=None, include_top=False)

f4_x = GlobalAveragePooling2D()(f4_base.output)

# # #Regularization with noise
f4_x = GaussianNoise(0.1)(f4_x)

f4_x = Dense(2048, activation='relu')(f4_x)
f4_x = Dense(5, activation='softmax')(f4_x)
model_4 = Model(inputs=[f4_base.input],outputs=[f4_x])

print(model_4.summary())
# # print(f2_base.summary())

In [ ]:
ls -l

In [ ]:
#use pickle to load model weights
import os
import pickle

pklfile= os.path.join('Cassava_NonGrp_S1_SENet154_LRG501_STD.pkl')

f= open(pklfile, 'rb')     #Python 3                 
weigh= pickle.load(f);                
f.close();

# restoredmodel= mymodel()
#use set_weights to load the modelweights into the model architecture
model_4.set_weights(weigh)

In [ ]:
for layer in model_4.layers[:-4]:
    layer.trainable=False
# for layer in model.layers[87:]:
#     layer.trainable=True)
for i,layer in enumerate(model_4.layers):
    print(i,layer.name,layer.trainable)

In [ ]:
#Non-Groups
#Split training and validation
#Using Expert Data

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras.optimizers import Adam
# from keras.utils import multi_gpu_model

import time, os
from math import ceil
import multiprocessing

train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
#     rescale = 1./255
    preprocessing_function=preprocess_input
)

NUM_GPU = 4
batch_size = 80

train_set = train_datagen.flow_from_directory('train',
                                                 target_size = (501, 501),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

valid_set = test_datagen.flow_from_directory('valid',
                                                 target_size = (501, 501),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

model_txt = 'bi-lstm'
# Helper: Save the model.
savedfilename = os.path.join('checkpoints', 'Cassava_NonGrp_S1_SENet154_LRG501_STD.hdf5')

checkpointer = ModelCheckpoint(savedfilename,
                          monitor='val_acc', verbose=1, 
                          save_best_only=True, mode='max',save_weights_only=True)########

# Helper: TensorBoard
tb = TensorBoard(log_dir=os.path.join('cassava_output', 'logs', model_txt))

# Helper: Save results.
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('cassava_output', 'logs', model_txt + '-' + 'training-' + \
    str(timestamp) + '.log'))

from keras.callbacks import Callback
class EarlyStoppingByAccVal(Callback):
    def __init__(self, monitor='val_acc', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current >= self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping" % epoch)
            self.model.stop_training = True

earlystopping = EarlyStoppingByAccVal(monitor='val_acc', value=0.8850, verbose=1)

#Using multiple models if more than 1 GPU
if NUM_GPU != 1:
    model_mul = multi_gpu_model(model_4, gpus=NUM_GPU)

# print('Loading pretrained weights')
# if NUM_GPU != 1:
#     model_mul.load_weights(savedfilename)
# # else:
# #     model.load_weights(savedfilename)

epochs = 30##!!!
lr = 1e-5
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)

model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

step_size_train=ceil(train_set.n/train_set.batch_size)
step_size_valid=ceil(valid_set.n/valid_set.batch_size)
# step_size_test=ceil(testing_set.n//testing_set.batch_size)

result = model_mul.fit_generator(
    generator = train_set, 
    steps_per_epoch = step_size_train,
    validation_data = valid_set,
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
    callbacks=[earlystopping],
#     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
    verbose=1) 

In [ ]:
!wget https://github.com/qubvel/classification_models/releases/download/0.0.1/senet154_imagenet_1000_no_top.h5

In [ ]:
mv /home/bribeiro/Phong/Nat19/Cassava_NonGrp_S1_NASNetLarge_LRG501_STD.pkl Cassava_NonGrp_S1_NASNetLarge_LRG501_STD.pkl 

In [ ]:
from classification_models.keras import Classifiers
from keras.models import Model
# from keras.layers import concatenate
from keras.layers import Dense, GlobalAveragePooling2D, Input, Embedding, LSTM
# from keras.applications.inception_v3 import preprocess_input

from keras.layers import GaussianNoise

nasnet, preprocess_input = Classifiers.get('nasnetlarge')

f4_base = nasnet(input_shape=(501,501,3), weights=None, include_top=False)

f4_x = GlobalAveragePooling2D()(f4_base.output)

# # #Regularization with noise
f4_x = GaussianNoise(0.1)(f4_x)

f4_x = Dense(2048, activation='relu')(f4_x)
f4_x = Dense(5, activation='softmax')(f4_x)
model_4 = Model(inputs=[f4_base.input],outputs=[f4_x])

print(model_4.summary())
# # print(f2_base.summary())

In [ ]:
#use pickle to load model weights
import os
import pickle

pklfile= os.path.join('Cassava_NonGrp_S1_NASNetLarge_LRG501_STD.pkl')

f= open(pklfile, 'rb')     #Python 3                 
weigh= pickle.load(f);                
f.close();

# restoredmodel= mymodel()
#use set_weights to load the modelweights into the model architecture
model_4.set_weights(weigh)

In [ ]:
#Non-Groups
#Split training and validation
#Using Expert Data

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras.optimizers import Adam
# from keras.utils import multi_gpu_model

import time, os
from math import ceil
import multiprocessing

train_datagen = ImageDataGenerator(
#     rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,##
#     brightness_range=[0.5, 1.5],##
    channel_shift_range=10,##
    fill_mode='nearest',
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
#     rescale = 1./255
    preprocessing_function=preprocess_input
)

NUM_GPU = 4
batch_size = 12

train_set = train_datagen.flow_from_directory('train',
                                                 target_size = (501, 501),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=True,
                                                 seed=7,
#                                                  subset="training"
                                              )

valid_set = test_datagen.flow_from_directory('valid',
                                                 target_size = (501, 501),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 shuffle=False,
                                                 seed=7,
#                                                  subset="validation"
                                             )

model_txt = 'bi-lstm'
# Helper: Save the model.
savedfilename = os.path.join('checkpoints', 'Cassava_NonGrp_S1_NASNetLarge_LRG501_STD.hdf5')

checkpointer = ModelCheckpoint(savedfilename,
                          monitor='val_acc', verbose=1, 
                          save_best_only=True, mode='max',save_weights_only=True)########

# Helper: TensorBoard
tb = TensorBoard(log_dir=os.path.join('cassava_output', 'logs', model_txt))

# Helper: Save results.
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('cassava_output', 'logs', model_txt + '-' + 'training-' + \
    str(timestamp) + '.log'))

from keras.callbacks import Callback
class EarlyStoppingByAccVal(Callback):
    def __init__(self, monitor='val_acc', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current >= self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping" % epoch)
            self.model.stop_training = True

earlystopping = EarlyStoppingByAccVal(monitor='val_acc', value=0.8850, verbose=1)

#Using multiple models if more than 1 GPU
if NUM_GPU != 1:
    model_mul = multi_gpu_model(model_4, gpus=NUM_GPU)

# print('Loading pretrained weights')
# if NUM_GPU != 1:
#     model_mul.load_weights(savedfilename)
# # else:
# #     model.load_weights(savedfilename)

epochs = 30##!!!
lr = 1e-5
decay = lr/epochs
optimizer = Adam(lr=lr, decay=decay)

model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

step_size_train=ceil(train_set.n/train_set.batch_size)
step_size_valid=ceil(valid_set.n/valid_set.batch_size)
# step_size_test=ceil(testing_set.n//testing_set.batch_size)

result = model_mul.fit_generator(
    generator = train_set, 
    steps_per_epoch = step_size_train,
    validation_data = valid_set,
    validation_steps = step_size_valid,
    shuffle=True,
    epochs=epochs,
    callbacks=[earlystopping],
#     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
    verbose=1) 

In [ ]:
#MUL 2 - Inception
#501
import keras

# from keras_applications.resnext import ResNeXt101, preprocess_input
# from keras.applications.resnet50 import ResNet50
# from keras.applications import InceptionV3
# from keras.applications import Xception
# from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from classification_models.senet import SEResNet101, preprocess_input
# from classification_models.keras import Classifiers


from keras.models import Model
from keras.layers import concatenate
from keras.layers import Dense, GlobalAveragePooling2D, Input, Embedding, LSTM
# from keras.applications.inception_v3 import preprocess_input

from keras.layers import GaussianNoise

# ResNext101, preprocess_input = Classifiers.get('resnext101')
# NASNetLarge = preprocess_input = Classifiers.get('nasnetlarge')

# SENet154, preprocess_input = Classifiers.get('senet154')

f2_base = SEInceptionResNetV2(include_top=True,
                        weights=None,
                        input_tensor=None,
                        input_shape=(501,501,3),
                        pooling=None)

# # f2_base = Classifiers.get('nasnetlarge')[0]((501, 501, 3), weights=None)
# f2_base = SENet154((501, 501, 3), weights=None)
# # f2_base = SeNet154((501,501,3), weights = 'imagenet', include_top=False) 
# # f2_base = ResNext101((501,501,3), weights = 'imagenet') 
# # # f2_base = NASNetLarge((501,501,3), weights = 'imagenet')

# # # for layer in f3_base.layers:
# # #     layer.trainable = False 
f2_x = f2_base.layers[-2].output
# # # f2_x = GlobalAveragePooling2D()(f2_x)

# # #Regularization with noise
# f2_x = GaussianNoise(0.1)(f2_x)

f2_x = Dense(2048, activation='relu')(f2_x)
f2_x = Dense(5, activation='softmax')(f2_x)
model_2 = Model(inputs=[f2_base.input],outputs=[f2_x])

print(model_2.summary())
# print(f2_base.summary())

In [ ]:
!pip install pretrainedmodels

In [ ]:
import pretrainedmodels

In [ ]:
print(pretrainedmodels.model_names)

In [ ]:
from classification_models.keras import Classifiers
from keras.models import Model
# from keras.layers import concatenate
from keras.layers import Dense, GlobalAveragePooling2D, Input, Embedding, LSTM
# from keras.applications.inception_v3 import preprocess_input

from keras.layers import GaussianNoise

senet, preprocess_input = Classifiers.get('seresnext101')

f2_base = senet(input_shape=(501,501,3), weights='imagenet', include_top=False)
f2_x = GlobalAveragePooling2D()(f2_base.output)

# # #Regularization with noise
f2_x = GaussianNoise(0.1)(f2_x)

f2_x = Dense(2048, activation='relu')(f2_x)
f2_x = Dense(5, activation='softmax')(f2_x)
model_2 = Model(inputs=[f2_base.input],outputs=[f2_x])

print(model_2.summary())
# # print(f2_base.summary())

In [ ]:
for layer in model_2.layers[:-3]:
    layer.trainable=False
# for layer in model.layers[87:]:
#     layer.trainable=True)
for i,layer in enumerate(model_2.layers):
    print(i,layer.name,layer.trainable)

In [ ]:
from keras.utils import to_categorical
import numpy as np
import random
import csv
import glob
import os
import os.path

def get_data():
    """Load data from file."""
    with open(os.path.join('cassava_data_senet.csv'), 'r') as fin:
        reader = csv.reader(fin)
        data = list(reader)

    return data

def get_extracted_sequence(sample):
    """Get the extracted features."""
    filename = sample[0] + '_' + sample[1] + '_' + sample[2]
    path = os.path.join('sequences', filename + '.npy')
    #print(path)
    if os.path.isfile(path):
        return np.load(path)
    else:
        return None
    
def get_classes():
    """Extract the classes from data"""
    classes = []
    for item in data:
        if item[1] not in classes:
            classes.append(item[1])

    # Sort them.
    classes = sorted(classes)

    # Return.
    return classes

def get_class_one_hot(class_str):
    """Encode and one-hot classes for training.
    Given a class as a string, return its number in the classes list. """
    # Encode
    label_encoded = classes.index(class_str)

    # One-hot
    label_hot = to_categorical(label_encoded, len(classes))

    assert len(label_hot) == len(classes)

    return label_hot

def get_all_sequences_in_memory(train_test):
    """
    Load everything into memory
    """
    # Get the right dataset.
    train, test = split_train_test()
    data = train if train_test == 'train' else test
    
    print("Loading %d samples into memory for %sing." % (len(data), train_test))
    
    X, y = [], []
    for row in data:
        sequence = get_extracted_sequence(row)
        # modify for single sequence
        sequence = np.expand_dims(sequence, axis=0)

        if sequence is None:
            print("Can't find sequence?")
            raise

        X.append(sequence)
        y.append(get_class_one_hot(row[1]))

    return np.array(X), np.array(y)

def split_train_test():
    """Split the data into train and test sets."""
    train = []
    test = []
    for item in data:
        if item[0] == 'train':
            train.append(item)
        else:
            test.append(item)
    return train, test

In [ ]:
from classification_models.keras import Classifiers
from keras.models import Model
# from keras.layers import concatenate
from keras.layers import Dense, GlobalAveragePooling2D, Input, Embedding, LSTM
# from keras.applications.inception_v3 import preprocess_input

from keras.layers import GaussianNoise

senet, preprocess_input = Classifiers.get('seresnext101')

f3_base = senet(input_shape=(501,501,3), weights='imagenet', include_top=False)
f3_x = GlobalAveragePooling2D()(f3_base.output)

# # # #Regularization with noise
# f2_x = GaussianNoise(0.1)(f2_x)

# f2_x = Dense(2048, activation='relu')(f2_x)
# f2_x = Dense(5, activation='softmax')(f2_x)
model_3 = Model(inputs=[f3_base.input],outputs=[f3_x])

print(model_3.summary())
# # print(f2_base.summary())

In [ ]:
from keras.preprocessing import image as imgload
# from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model
from keras.layers import Input
import numpy as np

class Extractor_SeNet():
    def __init__(self, weights=None):

        # assign model_1 (Xception)
        self.model = model_3
            
    def extract(self, image_path):
        img = imgload.load_img(image_path, target_size=(501, 501)) #input size
        #img = imgload.load_img(image_path, target_size=(224, 224)) #densenet, mobilenet
        x = imgload.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        # Get the prediction.
        features = self.model.predict(x)

#         if self.weights is None:
#             # For imagenet/default network:
#             features = features[0]
#         else:
#             # For loaded network:
#             features = features[0]

        return features[0]

In [ ]:
mv sequences sequences_bkp

In [ ]:
rm -r sequences

In [ ]:
mkdir sequences

In [ ]:
# Feature Extraction
#

import numpy as np
import os.path
from tqdm import tqdm
import sys
import os

import csv
import glob
import os
import os.path

# Get model
model = Extractor_SeNet()

# Multiple GPUs
NUM_GPU = 4
model.model = multi_gpu_model(model.model, gpus=NUM_GPU)

# Load weights
# model.model.load_weights(os.path.join('checkpoints', 'Cassava_NonGrp_S1_Xception_LRG501_Mul_Pretrained_STD.hdf5'))

# Get the dataset.
data = get_data()
print(len(data))

# Get the model.
# model = Extractor_InceptionV3()

# Loop through data.
pbar = tqdm(total=len(data))
for images in data:
    feature_path = os.path.join('sequences', images[0] + '_' + images[1] + '_' + images[2])

    # Skip if the file already existed.
    if os.path.isfile(feature_path + '.npy'):
        pbar.update(1)
        continue

    image_path = os.path.join(images[0], images[1], images[2])
    feature = model.extract(image_path)

    # Save the feature.
    np.save(feature_path, feature)
    
    pbar.update(1)

pbar.close()

In [ ]:
from keras.utils import to_categorical
import numpy as np
import random

def get_extracted_sequences_one_model(dir1, sample):
    """Get the extracted features."""
    filename = sample[0] + '_' + sample[1] + '_' + sample[2]
    path1 = os.path.join(dir1, filename + '.npy')
    #print(path)
    if os.path.isfile(path1):
        return np.load(path1)
    else:
        return None
    
def get_all_sequences_one_model_in_memory(dir1, train_test):
    """
    Load everything into memory
    """
    # Get the right dataset.
    train, test = split_train_test()
    data = train if train_test == 'train' else test
    
    print("Loading %d samples into memory for %sing." % (len(data), train_test))
    
    X, y = [], []
    for row in data:
        sequence1 = get_extracted_sequences_one_model(dir1, row)
        
        # add two sequences
#         sequence = np.concatenate(sequence1, axis=0)
        # modify for single sequence
        sequence = np.expand_dims(sequence1, axis=0)

        if sequence is None:
            print("Can't find sequence?")
            raise

        X.append(sequence)
        y.append(get_class_one_hot(row[1]))

    return np.array(X), np.array(y)

In [ ]:
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D, LeakyReLU
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,
    MaxPooling2D)

def base_line_one_model():
    """Build a simple model"""
    # Model.
    model = Sequential()
#     model.add(Dense(32, input_shape=(16,)))
    model.add(Flatten(input_shape=(1,2048)))
#     model.add(Dense(1024, activation='relu'))#inceptionv3
#     model.add(Dense(2048, activation='relu'))
    model.add(Dense(1024))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))

    return model

In [ ]:
senet_model = base_line_one_model()
senet_model.summary()

In [ ]:
# Get the dataset.
data = get_data()
print(len(data))

# Get the classes.
classes = get_classes()
print(classes)

In [ ]:
#3 models
#IcpResNet, Inception, Xception
#501

from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
import time, os

model_txt = 'bi-lstm'
# Helper: Save the model.
savedfilename = os.path.join('checkpoints', 'Cassava_NonGrp_S1_TL_SeNet501_Mul_Pretrained_STD.hdf5')

checkpointer = ModelCheckpoint(savedfilename,
                          monitor='val_acc', verbose=1, 
                          save_best_only=True, mode='max',save_weights_only=False)

# Helper: TensorBoard
tb = TensorBoard(log_dir=os.path.join('cassava_output', 'logs', model_txt))

# Helper: Save results.
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('cassava_output', 'logs', model_txt + '-' + 'training-' + \
    str(timestamp) + '.log'))

from keras.callbacks import Callback
class EarlyStoppingByAccVal(Callback):
    def __init__(self, monitor='val_acc', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current >= self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping" % epoch)
            self.model.stop_training = True

early_stopper = EarlyStoppingByAccVal(monitor='val_acc', value=0.99, verbose=1)


X_train, y_train = get_all_sequences_one_model_in_memory('sequences','train')
X_valid, y_valid = get_all_sequences_one_model_in_memory('sequences','valid')

from keras.optimizers import Adam

NUM_GPU = 4
batch_size = 60
nb_epoch = 500

lr = 1e-5
decay = lr/nb_epoch
optimizer = Adam(lr=lr, decay=decay)

#Using multiple models if more than 1 GPU
if NUM_GPU != 1:
    mul_model = multi_gpu_model(senet_model, gpus=NUM_GPU)

if NUM_GPU != 1:
    mul_model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

result = mul_model.fit(
            X_train,
            y_train,
            batch_size=batch_size,
            validation_data=(X_valid, y_valid),
            verbose=1,
            callbacks=[tb, csv_logger, checkpointer, early_stopper],
            epochs=nb_epoch)